In [91]:
import codecs
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

Load data in Tensorflow.

In [92]:
root = "../"
training_data_folder = '%straining_data/web-radio/output/rec' % root
embDir = '%sembeddings' % root
what = 'artist'

uri_file = '%s/%s.emb.u' % (embDir, what)
vector_file = '%s/%s.emb.v' % (embDir, what)
# header_file = '%s/%s.emb.h' % (embDir, what)
training_file = '%s/%s.dat' % (training_data_folder, what)

vectors = np.array([line.strip().split(' ') for line in codecs.open(vector_file, 'r', 'utf-8')])
# heads = np.array([line.strip() for line in codecs.open(header_file, 'r', 'utf-8')])
uris = np.array([line.strip() for line in codecs.open(uri_file, 'r', 'utf-8')])

train_array = np.array([line.strip().split(' ') for line in codecs.open(training_file, 'r', 'utf-8')])
pd.DataFrame(train_array, columns=['seed', 'target', 'score']).head()


,seed,target,score
0,http://data.doremus.org/artist/d33ebb23-7b8d-3...,http://data.doremus.org/artist/6329cd86-d47a-3...,1
1,http://data.doremus.org/artist/01915146-b964-3...,http://data.doremus.org/artist/6329cd86-d47a-3...,1
2,http://data.doremus.org/artist/01915146-b964-3...,http://data.doremus.org/artist/d33ebb23-7b8d-3...,1
3,http://data.doremus.org/artist/72b3b303-5c15-3...,http://data.doremus.org/artist/6329cd86-d47a-3...,1
4,http://data.doremus.org/artist/72b3b303-5c15-3...,http://data.doremus.org/artist/d33ebb23-7b8d-3...,1


Data pre-processing: I want to substitute the seed and target with their embeddings

In [93]:
def get_embs(x):
    # uri to embedding
    v = vectors[np.argwhere(uris == x)]
    if v.size == 0:
        result = -2. * np.ones(vectors[0].size)
    else:
        result = v[0][0]
    return result.astype('float32')

In [94]:
col1 = np.array([get_embs(xi) for xi in train_array[:, 0]])
col2 = np.array([get_embs(xi) for xi in train_array[:, 1]])
col1 = np.concatenate((col1, [12., 45., 73.] * np.ones((train_array.shape[0], 3))), axis=1)
col2 = np.concatenate((col2, [12., 45., 73.] * np.ones((train_array.shape[0], 3))), axis=1)
col3 = np.array(train_array[:, 2]).astype('float32')
col3 = col3.reshape((col3.size, 1))

In [95]:
def next_batch(num, data, labels):
    """
    Return a total of `num` random samples and labels. 
    """
    idx = np.arange(0, len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = data[idx]
    labels_shuffle = labels[idx]
    return data_shuffle, labels_shuffle

int

In [170]:
# training_vector = np.concatenate((col1, col2, col3), axis=1)
b_size = 4
x_size = col1[0].shape[0]

training_vector = []
for i in range(0,10000):
    l, b = next_batch(4, np.concatenate((col1, col2), axis=1), col3)
    training_vector.append(np.concatenate([b[:,:x_size], b[:,x_size:], l], axis=1).flatten())
    
training_vector = np.array(training_vector)
training_vector.shape

(10000, 140)

Split test and train

In [171]:
len(train_label)

7000

In [172]:
train, test = train_test_split(training_vector, train_size=0.7)

train_vector = train[:, : -b_size]
train_label = train[:, -b_size:]
# train_label = train_label.reshape((len(train_label), b_size))

test_vector = test[:, :-b_size]
test_label = test[:, -b_size:]
# test_label = test_label.reshape((len(test_label), b_size))

/Users/pasquale/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [173]:
print('Train')
print(train_vector.shape)
print(train_label.shape)
print('Test')
print(test_vector.shape)
print(test_label.shape)

Train
(7000, 136)
(7000, 4)
Test
(3000, 136)
(3000, 4)


In [174]:
# Parameters
learning_rate = 0.1
num_steps = 1000
batch_size = 64
display_step = 100

In [175]:
# Network Parameters
n_hidden_1 = 256  # 1st layer number of neurons
n_hidden_2 = 256  # 2nd layer number of neurons
num_input = train_vector[0].size
num_output = col1[0].size
num_output_wrap = train_label[0].size

# tf Graph input
X = tf.placeholder(tf.float32, [None, num_input], name="X")
Y = tf.placeholder(tf.float32, [None, num_output_wrap], name="Y")

Neural network

In [176]:
# Create model
def neural_net(x):
    with tf.name_scope('hidden_1') as scope:
        # Hidden fully connected layer with 256 neurons
        w1 = tf.Variable(tf.random_normal([num_input, n_hidden_1]), name='w')
        b1 = tf.Variable(tf.random_normal([n_hidden_1]), name='b')
        layer_1 = tf.add(tf.matmul(x, w1), b1, name='o')
    with tf.name_scope('hidden_2') as scope:
        # Hidden fully connected layer with 256 neurons
        w2 = tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]), name='w')
        b2 = tf.Variable(tf.random_normal([n_hidden_2]), name='b')
        layer_2 = tf.add(tf.matmul(layer_1, w2), b2, name='o')
    with tf.name_scope('out_layer') as scope:
        # Output fully connected layer with a neuron for each class
        wo = tf.Variable(tf.random_normal([n_hidden_2, num_output]), name='w')
        bo = tf.Variable(tf.random_normal([num_output]), name='b')
        out_layer = tf.add(tf.matmul(layer_2, wo), bo, name="o")
        with tf.name_scope('u_norm') as scope:
            row_sum = tf.reduce_sum(out_layer, axis=1, keepdims=True)
            return tf.divide(out_layer, row_sum)

In [177]:
def weighted_l2(a, b, w):
    with tf.name_scope('weighted_l2') as scope:
        # https://stackoverflow.com/a/8861999/1218213
        q = tf.subtract(a, b, name="q")
        # return np.sqrt((w * q * q).sum())
        pow_q = tf.cast(tf.pow(q, 2), tf.float32, name="q-power")

        return tf.reduce_sum(tf.multiply(w, pow_q), axis=1, name="o", keepdims=True)

In [178]:
def compute_penalty(expected, taken, total):
    with tf.name_scope('penalty') as scope:
        penalty = tf.divide(tf.subtract(expected, taken), total)
        return tf.cast(penalty, tf.float32)


def neural_net_wrap(x, previous_out):
    with tf.name_scope('nn_wrapper') as scope:
        lt = previous_out.shape.as_list()[0]  # vertical size of the tensor
        lh = previous_out[0].shape.as_list()[0]  # horizontal size of the tensor
        
        split_n = int(x[0].shape.as_list()[0]/(2*lh))
        seed, target = tf.split(x, split_n, axis=1)
        
        
        print(couples[0].shape)
        bs = tf.equal(seed, -2.)
        bt = tf.equal(target, -2.)

        _ones = tf.ones_like(previous_out, tf.float32)
        max_distance = weighted_l2(_ones, _ones * -1., previous_out)

        bad_mask = tf.logical_or(bs, bt)
        good_mask = tf.logical_not(bad_mask)

        bs_count = tf.count_nonzero(tf.logical_not(bs), axis=1, keepdims=True)
        good_count = tf.count_nonzero(good_mask, axis=1, keepdims=True)

        _zeros = tf.zeros_like(previous_out, tf.float32)
        _seed = tf.where(good_mask, seed, _zeros)
        _target = tf.where(good_mask, target, _zeros)

        # distance
        d = weighted_l2(_seed, _target, previous_out)

        # how much info I am not finding
        penalty = compute_penalty(bs_count, good_count, lh)
        multiplier = tf.subtract(1., penalty)
        
        # score
        s = tf.divide(tf.subtract(max_distance, d), max_distance)
        return tf.multiply(s, multiplier)

In [179]:
# Construct model
intermediate = neural_net(X)
logits = neural_net_wrap(X, intermediate)

(?, 34)


NameError: name 'seed' is not defined

In [60]:
logits.shape

TensorShape([Dimension(None), Dimension(1)])

In [52]:
# Define loss and optimizer
# loss_op = MSE
loss_op = tf.reduce_mean(tf.square(tf.subtract(logits, Y)))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.less(tf.subtract(logits, Y), 0.1)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


In [53]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [54]:
with tf.Session() as sess:
    writer = tf.summary.FileWriter("output", sess.graph)

    # Run the initializer
    sess.run(init)

    print("Start learning")
    for step in range(1, num_steps + 1):
        batch_x, batch_y = next_batch(batch_size, train_vector, train_label)

        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            preds, my_weights, loss, acc = sess.run([logits, intermediate, loss_op, accuracy],
                                                    feed_dict={X: batch_x, Y: batch_y})
            
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
            # print("Predictions %s VS %s" % (preds[0], batch_y[0]))
            np.set_printoptions(precision=2)
            print("My weights %s" % np.mean(my_weights, axis=0))

    print("Optimization Finished!")

    print("Testing Accuracy:",
          sess.run(accuracy, feed_dict={X: test_vector, Y: test_label}))
    writer.close()


Start learning
Step 1, Minibatch Loss= 10790.4355, Training Accuracy= 1.000
My weights [0.01 0.01 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02
 0.02 0.02 0.01 0.01 0.01 0.01 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02
 0.02 0.02 0.02 0.02 0.02 0.01 0.01 0.01 0.01 0.02 0.02 0.02 0.02 0.02
 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.01 0.01 0.01 0.02
 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02]
Step 100, Minibatch Loss= 11132.3525, Training Accuracy= 1.000
My weights [0.01 0.01 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02
 0.02 0.02 0.01 0.01 0.01 0.01 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02
 0.02 0.02 0.02 0.02 0.02 0.01 0.01 0.01 0.01 0.02 0.02 0.02 0.02 0.02
 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.01 0.01 0.01 0.02
 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02]
Step 200, Minibatch Loss= 10656.1084, Training Accuracy= 1.000
My weights [0.01 0.01 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02
 0

ValueError: Cannot feed value of shape (3000, 139) for Tensor 'X:0', which has shape '(?, 136)'